<a href="https://colab.research.google.com/github/caiphus24/load-shortfall-regression-predict-api/blob/master/sklearn_comet_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Comet ML  

Comet is a great tool for model versioning and experimentation as it records the parameters and conditions from each of your experiements- allowing you to reproduce your results, or go back to a previous version of your experiment.  

To create an account, visit https://www.comet.ml/  
Follow the instructions for a single user account. Once that is created, you will see a project folder. That is where the records of your experiments can be viewed. 

Comet has an abundance of tutorials and scripts, we're just going to run through this notebook to get you started on the right track. For this illustration, we will be using one of the examples found on the Comet ML GitHub repo.

To begin with, you should install as illustrated below if you don't already have it. *Always import Experiment at the top of your notebook/script.*


In [ ]:
!pip install comet_ml

     |████████████████████████████████| 194kB 5.1MB/s 
     |████████████████████████████████| 204kB 16.6MB/s 
     |████████████████████████████████| 409kB 16.4MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=8c4ccbe8cbf474f180b1e55f2102ed6b5a849b8f6d8b65c359deb50756a1111d
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [ ]:
from comet_ml import Experiment

You will see an API key button at the top of the page when you click on an experiment- use this key as illustrated below to link your current workspace to comet. (If a project is empty, the code below will autogenerate for you on the project page, just copy and paste it in here)

In [ ]:
# Setting the API key (saved as environment variable)
experiment = Experiment(#api_key='your api key here'
                        project_name="general", workspace="jo-moon") 

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jo-moon/general/96b60794fd8747a084b2a1c0cc015a33



Import the rest of your necessary libraries as you usually would. For this demonstration we will be using the breast cancer dataset for classification so we will also import that from sklearn.

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [ ]:
# Have a look at your dataset
cancer = load_breast_cancer()
print("cancer.keys(): {}".format(cancer.keys()))
print("Shape of cancer data: {}\n".format(cancer.data.shape))
print("Sample counts per class:\n{}".format(
      {n: v for n, v in zip(cancer.target_names, np.bincount(cancer.target))}))
print("\nFeature names:\n{}".format(cancer.feature_names))

cancer.keys(): dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
Shape of cancer data: (569, 30)

Sample counts per class:
{'malignant': 212, 'benign': 357}

Feature names:
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


Split your data into train and test sets, keep in mind that you need to set a random state for your results to be reproduced!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data,
    cancer.target,
    stratify=cancer.target,
    random_state=7)

In [ ]:
# Scale your data!

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## GridSearch 

For this example we've used a gridsearch but you may use a model with default parameters or your own parameters too- Just remember to add/remove the neccesary data when you are logging your parameters at the end of the experiment.

The `param_grid` variable contains the 'C' values we want our gridsearch to iterate through.



In [ ]:
logreg = LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 100]}

In [ ]:
# Training and testing using GridSearch
clf = GridSearchCV(logreg,
                   param_grid=param_grid,
                   cv=10,
                   n_jobs=-1)

clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

## Results

Now that our model has trained, we can have a look at the results- Below is a confusion matrix indicating that at first glance, we have a fairly good model going. We then save the F1 score, Precision, and Recall as individual variables to go into our metric dictionary for logging.

P.S. have a look at the Comet tutorial page for interesting confusion matrix plots.

In [ ]:
print("\nResults\nConfusion matrix \n {}".format(
    confusion_matrix(y_test, y_pred)))


Results
Confusion matrix 
 [[52  1]
 [ 2 88]]


In [ ]:
# Saving each metric to add to a dictionary for logging

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [ ]:
# Create dictionaries for the data we want to log

params = {"random_state": 7,
          "model_type": "logreg",
          "scaler": "standard scaler",
          "param_grid": str(param_grid),
          "stratify": True
          }
metrics = {"f1": f1,
           "recall": recall,
           "precision": precision
           }

In [ ]:
# Log our parameters and results
experiment.log_parameters(params)
experiment.log_metrics(metrics)

If you're using comet within a jupyter notebook, it's important to end your experiment when you've finished as illustrated below.

In [ ]:
experiment.end()

COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/jo-moon/general/96b60794fd8747a084b2a1c0cc015a33
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     f1                      : (0.9832402234636872, 0.9832402234636872)
COMET INFO:     precision               : (0.9887640449438202, 0.9887640449438202)
COMET INFO:     recall                  : (0.9777777777777777, 0.9777777777777777)
COMET INFO:     sys.cpu.percent.01 [31] : (0.9, 10.3)
COMET INFO:     sys.cpu.percent.02 [31] : (0.9, 7.5)
COMET INFO:     sys.cpu.percent.avg [31]: (0.95, 8.9)
COMET INFO:     sys.ram.total [31]      : (13653561344.0, 13653561344.0)
COMET INFO:     sys.ram.used [31]       : (592932864.0, 735928320.0)
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


## Display  

Running `experiment.display()` will show you your experiments comet.ml page inside your notebook as illustrated below. You can do this immediately after an experiment is run, and logged. 

In [ ]:
experiment.display()